In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import xml.etree.ElementTree as xet

df = pd.read_csv('label.csv')
df.head()

fileName = df['filepath'][0]
def getFileName(fileName):
    file_name = xet.parse(fileName).getroot()[1].text
    return  os.path.join("./images_only",file_name)
getFileName(fileName)
imagesPaths = list(df['filepath'].apply(getFileName))
#imagesPaths
xmins = list(df['xmin'])
ymins = list(df['ymin'])
xmaxs = list(df['xmax'])
ymaxs = list(df['ymax'])
#xmins
print('k')

k


In [2]:
#fp = imagesPaths[0]
#img = cv2.imread(fp)


#cv2.imshow('example',img)
#cv2.waitkey(0)
#cv2.destroyAllWindows()
#if img is not None:
#    cv2.namedWindow('example',cv2.WINDOW_NORMAL)
#    cv2.imshow('Image', img)
#   cv2.waitKey(0)
#    cv2.destroyAllWindows()
#else:      
#    print("Error: Image not loaded.")



In [3]:
def loadAndShowImg(index):
    fp = imagesPaths[index]
    img = cv2.imread(fp)
    if img is not None:
        cv2.rectangle(img,(xmins[index],ymins[index]),(xmaxs[index],ymaxs[index]),(0,255,0),3)
        cv2.namedWindow('example',cv2.WINDOW_NORMAL)
        cv2.imshow('eample',img)
        cv2.waitKey(0)    
        cv2.destroyAllWindows()
    else:
         print("Error: Image not loaded.")

i = 0
def showNextImg(p):
    if p > len(ymaxs):
        cv2.destroyAllWindows()
        return
    else:
        p = p + 1
    
    print("Showing image.",p)
    loadAndShowImg(p)
    
    


#loadAndShowImg(1)
#cv2.waitKey(0)
#cv2.destroyAllWindows()




    



# Data Processing

In [4]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img , img_to_array

In [5]:
 labels = df.iloc[:,1:].values
#df.iloc[:,1:]
# labels

In [6]:

data = []
output = []
print('Imageggg  error')
for ind in range(len(imagesPaths)):
    image = imagesPaths[ind] 
    img_arr = cv2.imread(image)
    if img_arr is None:
        print('Image error')
        break
    else:
        print('processing image ,',ind)
    h,w,d = img_arr.shape
    #processing
    load_image = load_img(image,target_size=(224,224))
    load_img_arr = img_to_array(load_image)
    norm_load_image_arr = load_img_arr/225.0 # normalize array data
    xmin,ymin,xmax,ymax=labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm= (nxmin , nxmax , nymin , nymax)
    data.append(norm_load_image_arr)
    output.append(label_norm)

output
data

Imageggg  error
processing image , 0
processing image , 1
processing image , 2
processing image , 3
processing image , 4
processing image , 5
processing image , 6
processing image , 7
processing image , 8
processing image , 9


[array([[[0.94666666, 1.1066667 , 1.0977777 ],
         [0.95111114, 1.1333333 , 1.1288888 ],
         [0.95111114, 1.1333333 , 1.1288888 ],
         ...,
         [0.6533333 , 0.50222224, 0.4977778 ],
         [0.6488889 , 0.51111114, 0.50222224],
         [0.72888887, 0.59555554, 0.58666664]],
 
        [[1.0355556 , 1.1333333 , 1.1333333 ],
         [1.        , 1.1333333 , 1.1333333 ],
         [0.9866667 , 1.1288888 , 1.1244445 ],
         ...,
         [0.6622222 , 0.50222224, 0.50222224],
         [0.5911111 , 0.44      , 0.44444445],
         [0.62222224, 0.47111112, 0.47555557]],
 
        [[1.0222223 , 1.12      , 1.1066667 ],
         [1.0177778 , 1.1333333 , 1.1288888 ],
         [1.        , 1.1333333 , 1.1333333 ],
         ...,
         [0.62666667, 0.4488889 , 0.45333335],
         [0.64      , 0.46666667, 0.48      ],
         [0.61333334, 0.45333335, 0.46222222]],
 
        ...,
 
        [[1.1333333 , 1.1333333 , 1.1333333 ],
         [1.1333333 , 1.1333333 , 1.12444

In [7]:
x = np.array(data,dtype=np.float32)
y=np.array(output ,dtype=np.float32)
x.shape,y.shape

((10, 224, 224, 3), (10, 4))

In [8]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((8, 224, 224, 3), (2, 224, 224, 3), (8, 4), (2, 4))

## Deep learning Model

In [9]:
from tensorflow.keras.applications import MobileNetV2 , InceptionV3 , InceptionResNetV2
from tensorflow.keras.layers import Dense , Dropout , Flatten , Input
from tensorflow.keras.models import Model
import tensorflow as tf


In [10]:
inception_resnet = InceptionResNetV2(weights = "imagenet",include_top = False ,
                                     input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable= False
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500, activation='relu')(headmodel)
headmodel = Dense(250, activation='relu')(headmodel)
headmodel = Dense(4, activation='sigmoid')(headmodel)

model = Model(inputs=inception_resnet.input , outputs = headmodel)



In [11]:
model.compile(loss='muse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0][0

 conv2d_11 (Conv2D)          (None, 25, 25, 64)           12288     ['average_pooling2d[0][0]']   
                                                                                                  
 batch_normalization_5 (Bat  (None, 25, 25, 96)           288       ['conv2d_5[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 batch_normalization_7 (Bat  (None, 25, 25, 64)           192       ['conv2d_7[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 batch_normalization_10 (Ba  (None, 25, 25, 96)           288       ['conv2d_10[0][0]']           
 tchNormalization)                                                                                
          

 block35_1_conv (Conv2D)     (None, 25, 25, 320)          41280     ['block35_1_mixed[0][0]']     
                                                                                                  
 custom_scale_layer (Custom  (None, 25, 25, 320)          0         ['mixed_5b[0][0]',            
 ScaleLayer)                                                         'block35_1_conv[0][0]']      
                                                                                                  
 block35_1_ac (Activation)   (None, 25, 25, 320)          0         ['custom_scale_layer[0][0]']  
                                                                                                  
 conv2d_21 (Conv2D)          (None, 25, 25, 32)           10240     ['block35_1_ac[0][0]']        
                                                                                                  
 batch_normalization_21 (Ba  (None, 25, 25, 32)           96        ['conv2d_21[0][0]']           
 tchNormal

                                                                                                  
 activation_25 (Activation)  (None, 25, 25, 32)           0         ['batch_normalization_25[0][0]
                                                                    ']                            
                                                                                                  
 activation_28 (Activation)  (None, 25, 25, 48)           0         ['batch_normalization_28[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_24 (Conv2D)          (None, 25, 25, 32)           10240     ['block35_2_ac[0][0]']        
                                                                                                  
 conv2d_26 (Conv2D)          (None, 25, 25, 32)           9216      ['activation_25[0][0]']       
          

                                                                                                  
 activation_32 (Activation)  (None, 25, 25, 32)           0         ['batch_normalization_32[0][0]
                                                                    ']                            
                                                                                                  
 activation_35 (Activation)  (None, 25, 25, 64)           0         ['batch_normalization_35[0][0]
                                                                    ']                            
                                                                                                  
 block35_4_mixed (Concatena  (None, 25, 25, 128)          0         ['activation_30[0][0]',       
 te)                                                                 'activation_32[0][0]',       
                                                                     'activation_35[0][0]']       
          

...

 e)                                                                  'activation_170[0][0]']      
                                                                                                  
 block8_2_conv (Conv2D)      (None, 5, 5, 2080)           933920    ['block8_2_mixed[0][0]']      
                                                                                                  
 custom_scale_layer_31 (Cus  (None, 5, 5, 2080)           0         ['block8_1_ac[0][0]',         
 tomScaleLayer)                                                      'block8_2_conv[0][0]']       
                                                                                                  
 block8_2_ac (Activation)    (None, 5, 5, 2080)           0         ['custom_scale_layer_31[0][0]'
                                                                    ]                             
                                                                                                  
 conv2d_17

 )                                                                  ]']                           
                                                                                                  
 block8_4_mixed (Concatenat  (None, 5, 5, 448)            0         ['activation_175[0][0]',      
 e)                                                                  'activation_178[0][0]']      
                                                                                                  
 block8_4_conv (Conv2D)      (None, 5, 5, 2080)           933920    ['block8_4_mixed[0][0]']      
                                                                                                  
 custom_scale_layer_33 (Cus  (None, 5, 5, 2080)           0         ['block8_3_ac[0][0]',         
 tomScaleLayer)                                                      'block8_4_conv[0][0]']       
                                                                                                  
 block8_4_

 )                                                                  ]']                           
                                                                                                  
 activation_186 (Activation  (None, 5, 5, 256)            0         ['batch_normalization_186[0][0
 )                                                                  ]']                           
                                                                                                  
 block8_6_mixed (Concatenat  (None, 5, 5, 448)            0         ['activation_183[0][0]',      
 e)                                                                  'activation_186[0][0]']      
                                                                                                  
 block8_6_conv (Conv2D)      (None, 5, 5, 2080)           933920    ['block8_6_mixed[0][0]']      
                                                                                                  
 custom_sc

 atchNormalization)                                                                               
                                                                                                  
 activation_191 (Activation  (None, 5, 5, 192)            0         ['batch_normalization_191[0][0
 )                                                                  ]']                           
                                                                                                  
 activation_194 (Activation  (None, 5, 5, 256)            0         ['batch_normalization_194[0][0
 )                                                                  ]']                           
                                                                                                  
 block8_8_mixed (Concatenat  (None, 5, 5, 448)            0         ['activation_191[0][0]',      
 e)                                                                  'activation_194[0][0]']      
          

 atchNormalization)                                                                               
                                                                                                  
 batch_normalization_202 (B  (None, 5, 5, 256)            768       ['conv2d_202[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_199 (Activation  (None, 5, 5, 192)            0         ['batch_normalization_199[0][0
 )                                                                  ]']                           
                                                                                                  
 activation_202 (Activation  (None, 5, 5, 256)            0         ['batch_normalization_202[0][0
 )                                                                  ]']                           
          